# DBT CLI on Cloud Run

This notebook demonstrates running Dataform using its command line interface (CLI) application, on Cloud Run. The steps are:

* Installation of local software
* Google Cloud configuration (permissions, enabling APIs)
* Local set-up of DBT and definitions of transformations
* Generation of container to execute the DBT application using Cloud Build
* Deploy the container to Cloud Run
* Execute the container in a Cloud Run Job

### Not discussed / out of scope

* Integrate the Cloud Build to update the container in a CI/CD process (we have not included the trigger)
* Orchestation of the Cloud Run Job: What triggers it?

For the orchestration, the Cloud Run Job is triggered by an HTTP request. This HTTP request can be launched from Composer (managed Airflow) as part of a DAG, Cloud Scheduler as part of cron-like schedule, or through some other mechanism that could run `gcloud beta run jobs execute`. 

### Installation of local software


In [ ]:
!pip install --user --upgrade dbt-core dbt-bigquery
!echo "export PATH=$(python -m site --user-base)/bin:$PATH" >> ~/.bashrc

In [ ]:
%%bash
source ~/.bashrc
dbt --version

### Enable Google Cloud APIs that we will need

In [ ]:
!gcloud services enable artifactregistry.googleapis.com cloudbuild.googleapis.com datacatalog.googleapis.com datalineage.googleapis.com run.googleapis.com

### Retrieve Google Cloud log-in credentials

Since this command requires some interaction, we will run it in a terminal: `gcloud auth login`

### DBT project set-up


In [ ]:
%%bash
source ~/.bashrc
mkdir -p ~/.dbt

cat << EOF > ~/.dbt/profiles.yml
dbt_proj_dir: # this needs to match the name of the project that you will init later
  target: dev
  outputs:
    dev:
      type: bigquery
      method: oauth
      project: $(gcloud config get project)
      dataset: dbt_demo
      threads: 1
      timeout_seconds: 300
      location: europe-west4
      priority: interactive
EOF


In [ ]:
!cat ~/.dbt/profiles.yml

In [ ]:
%%bash
source ~/.bashrc
cd dbt_proj_dir
dbt debug

In [ ]:
%%bash
source ~/.bashrc
yes N | dbt init dbt_proj_dir

If this works, copy over the profile config to the current project directory, so we include it in the container later:

In [ ]:
!cp ~/.dbt/profiles.yml dbt_proj_dir/

### Prepare some sample data in BigQuery

Start by creating the datasets, one for the raw data, and one for the data transformed with Dataform:

In [ ]:
!bq --location=europe-west4 mk --dataset ${PROJECT_ID}.prod_raw
!bq --location=europe-west4 mk --dataset ${PROJECT_ID}.dbt_demo

Upload the raw data:

In [ ]:
!bq load --source_format=PARQUET prod_raw.sales_data data/sales.parquet

### DBT definitions and transformations

First we declare the source table:

In [ ]:
!mkdir -p dbt_proj_dir/models/sales

In [ ]:
%%writefile dbt_proj_dir/models/sales/sources.yml
version: 2

sources:
  - name: prod_raw
    tables:
      - name: sales_data
        description: "Ingested sales data"

In [ ]:
%%writefile dbt_proj_dir/models/sales/sales_agg.sql
{{
  config(
    materialized = "table"
  )
}}

WITH daily_orders AS (
    SELECT
      DATE(orderdate) AS order_date, 
      PRODUCTLINE AS product_line,
      ROUND(SUM(SALES), 1) AS sales_value
    FROM
      {{ source("prod_raw", "sales_data") }}
    WHERE
      STATUS = "Shipped"
    GROUP BY
      1,
      2
)

SELECT 
    order_date, 
    product_line, 
    sales_value, 
    ROUND(SUM(sales_value) OVER (ORDER BY DATE(order_date) ROWS BETWEEN 7 PRECEDING AND CURRENT ROW  ), 1) AS rolling_average
FROM daily_orders
ORDER BY 1 DESC


### Test running the transformations

The expected output is:

```
17:45:56  Running with dbt=1.3.1

17:45:56  Found 2 models, 0 tests, 0 snapshots, 0 analyses, 319 macros, 0 operations, 0 seed files, 1 source, 0 exposures, 0 metrics
17:45:56  
17:45:57  Concurrency: 1 threads (target='dev')
17:45:57  
17:45:57  1 of 2 START sql table model dbt_demo.sales_agg ................................ [RUN]
17:45:59  1 of 2 OK created sql table model dbt_demo.sales_agg ........................... [CREATE TABLE (560.0 rows, 943.4 KB processed) in 2.37s]
17:45:59  2 of 2 START sql table model dbt_demo.sales_agg-checkpoint ..................... [RUN]
17:46:01  2 of 2 OK created sql table model dbt_demo.sales_agg-checkpoint ................ [CREATE TABLE (560.0 rows, 943.4 KB processed) in 2.12s]
17:46:01  
17:46:01  Finished running 2 table models in 0 hours 0 minutes and 4.85 seconds (4.85s).
17:46:01  
17:46:01  Completed successfully
17:46:01  
17:46:01  Done. PASS=2 WARN=0 ERROR=0 SKIP=0 TOTAL=2
```

In [ ]:
!dbt run --project-dir dbt_proj_dir/

### Build the container to run on Cloud Run

The container will have the Dataform application and our transformation code.

First we create a repository in Artifact Registry to store the container:

In [ ]:
!gcloud config set artifacts/location europe-west4

In [ ]:
!gcloud artifacts repositories create dbt --repository-format=docker

Define a `Dockerfile` to generate the image:

In [ ]:
%%writefile dbt_proj_dir/Dockerfile
# Generic Dockerfile from https://github.com/dbt-labs/dbt-core/blob/main/docker/Dockerfile

# Top level build args
ARG build_for=linux/amd64

##
# base image (abstract)
##
FROM --platform=$build_for python:3.10.7-slim-bullseye as base

# N.B. The refs updated automagically every release via bumpversion
# N.B. dbt-postgres is currently found in the core codebase so a value of dbt-core@<some_version> is correct

ARG dbt_core_ref=dbt-core@v1.4.0b1
ARG dbt_bigquery_ref=dbt-bigquery@v1.4.0b1
# special case args
ARG dbt_spark_version=all
ARG dbt_third_party

# System setup
RUN apt-get update \
  && apt-get dist-upgrade -y \
  && apt-get install -y --no-install-recommends \
    git \
    ssh-client \
    software-properties-common \
    make \
    build-essential \
    ca-certificates \
    libpq-dev \
  && apt-get clean \
  && rm -rf \
    /var/lib/apt/lists/* \
    /tmp/* \
    /var/tmp/*

# Env vars
ENV PYTHONIOENCODING=utf-8
ENV LANG=C.UTF-8

# Update python
RUN python -m pip install --upgrade pip setuptools wheel --no-cache-dir

##
# dbt-core
##
FROM base as dbt-core
RUN python -m pip install --no-cache-dir "git+https://github.com/dbt-labs/${dbt_core_ref}#egg=dbt-core&subdirectory=core"

##
# dbt-bigquery
##
FROM base as dbt-bigquery
RUN python -m pip install --no-cache-dir "git+https://github.com/dbt-labs/${dbt_bigquery_ref}#egg=dbt-bigquery"

##
# dbt-third-party
##
FROM dbt-core as dbt-third-party
RUN python -m pip install --no-cache-dir "${dbt_third_party}"

##
# dbt-all
##
FROM base as dbt-all
RUN apt-get update \
  && apt-get dist-upgrade -y \
  && apt-get install -y --no-install-recommends \
    python-dev \
    libsasl2-dev \
    gcc \
    unixodbc-dev \
  && apt-get clean \
  && rm -rf \
    /var/lib/apt/lists/* \
    /tmp/* \
    /var/tmp/*
  RUN python -m pip install --no-cache "git+https://github.com/dbt-labs/${dbt_bigquery_ref}#egg=dbt-bigquery"


# Set working directory
ENV DBT_DIR /dbt/
WORKDIR $DBT_DIR

# Copy files to the image
COPY . $DBT_DIR

# Install Dbt deps
RUN dbt deps

# Run dbt
ENTRYPOINT ["dbt", "run"]

Here is a configuration file for Cloud Build to run the Docker file. We don't strictly need this, but it gives us more options.

In [ ]:
%%writefile dbt_proj_dir/cloudbuild.yaml
steps:
- name: gcr.io/cloud-builders/docker
  id: Build DBT image
  env: 
    - 'DOCKER_BUILDKIT=1'
  args: [
      'build',
      '-t', 'europe-west4-docker.pkg.dev/${PROJECT_ID}/dbt/dbt-demo',
      '--cache-from', 'europe-west4-docker.pkg.dev/${PROJECT_ID}/dbt/dbt-demo:latest',
      '.'
    ]

- name: gcr.io/cloud-builders/docker
  id: Push DBT image to Artifact Registry
  args: [
      'push',
      'europe-west4-docker.pkg.dev/${PROJECT_ID}/dbt/dbt-demo:latest'
    ]

options:
  logging: CLOUD_LOGGING_ONLY

Submit the build to Cloud Build to generate the container and push it to Artifact Registry:

In [ ]:
!gcloud builds submit dbt_proj_dir --config=dbt_proj_dir/cloudbuild.yaml --region=europe-west4

### Test the container

Now you can test it:
    
```
docker pull europe-west4-docker.pkg.dev/${PROJECT_ID}/dataform/dataform-demo:latest
docker run europe-west4-docker.pkg.dev/${PROJECT_ID}/dataform/dataform-demo:latest
```

The output should be as before when you tested running the transformations.

### Create a Cloud Run Job for serverless execution


In [ ]:
import os
PROJECT_ID = os.popen('gcloud config get project').read()[:-1]

In [ ]:
!gcloud beta run jobs create dbt-demo --image europe-west4-docker.pkg.dev/$PROJECT_ID/dbt/dbt-demo:latest --region europe-west4

Run the Cloud Run job, which will run the Dataform transformations and recreate the aggregated table:

In [ ]:
!gcloud beta run jobs execute dbt-demo --region europe-west4

### CI/CD workflow using Github triggers

See the [Cloud Build documentation](https://cloud.google.com/build/docs/automating-builds/github/build-repos-from-github#gcloud).